<a href="https://colab.research.google.com/github/tufts-mathmodeling/HW/blob/master/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import cvxpy as cp

# Math Modeling Homework 4 (Spring 2020)

## Problem 1: Duality (16 points)

Consider the primal linear program

\begin{align*}
\text{maximize } ~~~11x_1 + 5x_2 & \\
\text{subject to } \phantom{~~11x_1 + 5x_2} &\\
x_1 + x_2 & \leq 7 \\
10x_1 + 4x_2 & \leq 40 \\
x_1, x_2 & \geq 0
\end{align*}

(a) Write the dual linear program. (See lecture notes.)

(b) Find the solutions to both the primal and the dual linear programs and
  plot the feasible sets.  
  
(c) Confirm that both the duality
  theorem and complementary slackness are satisfied.  What are the
  dual prices ("shadow prices") of each of the constraints?

(d) Does the dual price provide an accurate prediction of the
  increase in the primal objective function when the right-hand side
  of the first constraint is increased by 1?  By 2?  By 4?


## Problem 2: Branch and bound (16 points)

Consider the linear program
\begin{align*}
\text{maximize }\phantom{~~~}50x_1 + 6x_2 + 35x_3 + 60x_4 &\\
\text{subject to }\phantom{~~~50x_1 + 6x_2 + 35x_3 + 60x_4} &\\
~~24x_1 + 76x_2 + 43x_3 + 754x_4 &\leq 800 \\
~~755x_1 + 27x_2 + 33x_3 + 67x_4 &\leq 850
\end{align*}

(a) Restrict $x_1$, $x_2$, $x_3$, and $x_4$ to be integer variables
that can take only the values $0$ and $1$.  Perform a branch and bound
process *by hand* (in the greedy style shown in class) to find the optimal solution, explaining your choices for
which variables to branch on and where to prune the tree.  Draw the
branch and bound tree for your solution.  

(b) Check your hand-solved answer to (a) by using the complete enumeration code below and report how you confirmed your answer. While you're at it, use the code below to solve the integer program where the decision variables can be in $\{0,1,2\}$ or $\{0,1,2,3\}$.  

(c) Finally, there is an implementation of branch-and-bound at the bottom of the template, but it's not following quite the same protocol as we did in class.  Read through it and experiment with it and explain how its process of examining branches is different from the one we learned in class.  
 


In [0]:
x = cp.Variable(4)
constraints = [
    [24, 76, 43, 754] @ x <= 800,
    [755, 27, 33, 67] @ x <= 850,
]
objective = cp.Maximize([50, 6, 35, 60] @ x)

First, let's solve the LP without any integer constraints.

In [0]:
no_integer_constraints = cp.Problem(objective,
                                    constraints=constraints + [x >= 0, x <= 1])
print('with no integer constraints, value is {:.3f}'.format(no_integer_constraints.solve()))
print('x is', np.around(x.value, 3))

Now let's do a "brute-force" complete enumeration.

In [0]:
max_x = 1  
base = max_x + 1  #  number of integer values that can be taken by each decision variable
n_vals = base**x.size  # size of naive tree
best_sol = -np.inf  # initialize best_sol at negative infinity
best_sol_x = None

for idx in range(n_vals):
    x_eq = [int(c) for c in np.base_repr(idx, base=base).zfill(x.size)] # each time through, x_eq is a new vector like [0, 0, 0, 0]
    prob = cp.Problem(objective, constraints=constraints + [x == x_eq])  # outputs value of obj fcn at this x_eq, as long as it's feasible
    sol = prob.solve(solver='ECOS')  # use this solver!
    if prob.status == 'infeasible':
        print(x_eq, 'is infeasible!')
    else:
        print(x_eq, 'value is {:.2f}'.format(sol))
        if sol > best_sol:
            best_sol = sol
            best_sol_x = x_eq
print()
print('best x is', best_sol_x)
print('value is {:.2f}'.format(best_sol))

## Some branch and bound code

### A stack-based solution

In the model algorithm below, we use a stack (represented by Python's `collection.deque`) to keep track of all unexplored branches. 

In [0]:
from collections import deque

def is_integer(v):
    """Determines if a vector is approximately integer-valued."""
    return np.all(np.abs(np.round(v) - v) <= 1e-8)

def branches(parent, max_x):
    """Generates the immediate children of a branch.
    
    Branches are represented as tuples of variable length.
    For instance, if `parent` is (1, 0) and `max_x` is 2,
    this function generates [(1, 0, 0), (1, 0, 1), (1, 0, 2)].
    """
    return [(*parent, br) for br in range(max_x + 1)]

def prune(upper_bounds, branch_stack, cutoff):
    """Prunes all branches from the stack with upper bounds below `cutoff`."""
    for branch, bound in upper_bounds.items():
        if branch in branch_stack and bound <= cutoff:
            branch_stack.remove(branch)
            
def branch_and_bound(objective, constraints, x, max_x):
    """Finds integer solutions to an LP with a branch-and-bound algorithm."""
    x_bounds = [x >= 0, x <= max_x]
    branch_stack = deque([()])
    best_int = -np.inf
    best_int_x = None
    best_branch = None
    upper_bounds = {}
    branch_history = {}

    while branch_stack:
        branch = branch_stack.popleft()
        int_constraints = [x[idx] == val for idx, val in enumerate(branch)]
        prob = cp.Problem(objective,
                          constraints=constraints + int_constraints + x_bounds)
        sol = prob.solve(solver='ECOS')
        if prob.status == 'optimal':
            if is_integer(x.value):
                if sol > best_int:
                    # We've found the new best integer solution!
                    # Remove any branches guaranteed to do worse.
                    if best_branch is not None:
                        branch_history[best_branch] = ('pruned', best_int)
                    best_int = sol
                    best_int_x = x.value
                    best_branch = branch
                    prune(upper_bounds, branch_stack, sol)
                    branch_history[branch] = ('best', sol)
                else:
                    # We've found an integer solution, but it's not as good
                    # as the best one we've seen. Prune!
                    branch_history[branch] = ('pruned', sol)
            else:
                if sol > best_int:
                    # We've found a non-integer branch worth exploring; more
                    # branching is necessary to get to an integer solution.
                    children = branches(branch, max_x)
                    assert len(children[0]) <= x.size
                    for child in children:
                        upper_bounds[child] = sol
                    branch_stack += children
                    branch_history[branch] = ('non-integer', sol)
                else:
                    # We've found a non-integer solution that's worse than the
                    # best integer solution we've seen. Prune!
                    branch_history[branch] = ('pruned', sol)
        else:
            branch_history[branch] = ('infeasible', None)
    print('best branch is', best_branch)
    print('best integer x is', np.round(best_int_x, 4))
    print('value is {:.2f}'.format(best_int))
    print('evaluated {:d} LPs'.format(len(branch_history)))
    return branch_history

In [0]:
branch_and_bound(objective, constraints, x, 1)